In [ ]:
!pip install simplejson

In [236]:
import pandas as pd
import gzip
import simplejson
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re
import spacy
from string import punctuation
import numpy as np
from tqdm import tqdm
from sklearn.feature_extraction.text import TfidfVectorizer
import gensim.downloader

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
stopwords = stopwords.words('english')
nlp = spacy.load('en', disable=['parser', 'ner'])
punctuation = punctuation + '—–«»·№˚¼°-;&'

In [ ]:
!wget 'http://deepyeti.ucsd.edu/jianmo/amazon/categoryFiles/Luxury_Beauty.json.gz'
!wget 'http://deepyeti.ucsd.edu/jianmo/amazon/metaFiles2/meta_Luxury_Beauty.json.gz'

--2021-12-04 15:32:42--  http://deepyeti.ucsd.edu/jianmo/amazon/categoryFiles/Luxury_Beauty.json.gz
Resolving deepyeti.ucsd.edu (deepyeti.ucsd.edu)... 169.228.63.50
Connecting to deepyeti.ucsd.edu (deepyeti.ucsd.edu)|169.228.63.50|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 73193215 (70M) [application/octet-stream]
Saving to: ‘Luxury_Beauty.json.gz.2’

Luxury_Beauty.json. 100%[===================>]  69.80M  31.0MB/s    in 2.3s    

2021-12-04 15:32:44 (31.0 MB/s) - ‘Luxury_Beauty.json.gz.2’ saved [73193215/73193215]

--2021-12-04 15:32:44--  http://deepyeti.ucsd.edu/jianmo/amazon/metaFiles2/meta_Luxury_Beauty.json.gz
Resolving deepyeti.ucsd.edu (deepyeti.ucsd.edu)... 169.228.63.50
Connecting to deepyeti.ucsd.edu (deepyeti.ucsd.edu)|169.228.63.50|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7608964 (7.3M) [application/octet-stream]
Saving to: ‘meta_Luxury_Beauty.json.gz.2’

meta_Luxury_Beauty. 100%[===================>]   7.26M  1

#Обработка данных

In [ ]:
def parse(path):
  false = False
  true = True
  g = gzip.open(path, 'rb')
  for l in g:
    yield eval(l)

def getDF(path):
  i = 0
  df = {}
  for d in parse(path):
    df[i] = d
    i += 1
  return pd.DataFrame.from_dict(df, orient='index')

In [ ]:
parse('Luxury_Beauty.json.gz')

<generator object parse at 0x7f8874c1f7d0>

In [ ]:
reviews = getDF('Luxury_Beauty.json.gz')

In [ ]:
good_reviews = reviews[['reviewText', 'summary', 'asin']]

In [ ]:
parse('meta_Luxury_Beauty.json.gz')

<generator object parse at 0x7f884ef4c7d0>

In [ ]:
meta = getDF('meta_Luxury_Beauty.json.gz')

In [ ]:
good_meta = meta[['description', 'title', 'asin']]

In [ ]:
all_data['reviewText'] = all_data['reviewText'].replace(' ', np.NaN, inplace=True)
all_data['summary'] = all_data['summary'].replace(' ', np.NaN, inplace=True)
all_data['description'] = all_data['description'].replace(' ', np.NaN, inplace=True)
all_data['title'] = all_data['title'].replace(' ', np.NaN, inplace=True)

In [ ]:
all_data = pd.DataFrame()
all_data = pd.merge(good_reviews, good_meta, on='asin')
all_data['reviewText'] = all_data['reviewText'].fillna(np.nan)
all_data['summary'] = all_data['summary'].fillna(np.nan)
all_data['description'] = all_data['description'].fillna(np.nan)
all_data['title'] = all_data['title'].fillna(np.nan)
all_data['reviewText'] = all_data['reviewText'].dropna()
all_data['summary'] = all_data['summary'].dropna()
all_data['description'] = all_data['description'].dropna()
all_data['title'] = all_data['title'].dropna()
all_data = all_data[~all_data['description'].isna()]
all_data = all_data[~all_data['summary'].isna()]
all_data = all_data[~all_data['reviewText'].isna()]
all_data = all_data[~all_data['title'].isna()]

In [ ]:
desc = []
for description in all_data['description']:
    descriptions = ' '.join(description)
    desc.append(descriptions)
all_data['description'] = desc

In [ ]:
def preprocessing(data, *columns):
  for column in columns:
    print(column)
    all_obj = []
    for obj in tqdm(data[column]):
      lemmatized_obj = ' '.join([token.lemma_ for token in nlp(obj)])
      lemmatized_obj = lemmatized_obj.lower()
      for i in punctuation:
        lemmatized_obj = lemmatized_obj.replace(i, ' ')
        lemmatized_obj = lemmatized_obj.replace('  ', ' ')
      all_obj.append(lemmatized_obj)
    data[column] = all_obj
  return data

In [ ]:
data_preprocessed = preprocessing(all_data.groupby('asin').first().reset_index(), 'description', 'summary', 'reviewText', 'title')

description


100%|██████████| 12111/12111 [01:31<00:00, 132.15it/s]


summary


100%|██████████| 12111/12111 [00:27<00:00, 443.25it/s]


reviewText


100%|██████████| 12111/12111 [00:45<00:00, 268.47it/s]


title


100%|██████████| 12111/12111 [00:32<00:00, 376.74it/s]


In [ ]:
data_preprocessed

,asin,reviewText,summary,description,title
0,B00004U9V2,pron buy two of these 8 5 fl oz hand cream an...,dispenser do not work,after a long day of handle thorny situation pr...,crabtree amp evelyn gardener s ultra moisturis...
1,B0000531EN,pron have only use pron once so far so not su...,over price,if pron have not experience the pleasure of ba...,ahava bath salts
2,B0000532JH,pron add ingredient so pron be not 100 pure m...,product have get bad,rich black mineral mud harvest from the bank o...,ahava dead sea mineral mud 8 5 oz pack of 4
3,B00005A77F,this have get to be the very good soap for gar...,good soap go,this liquid soap with convenient pump dispense...,crabtree amp evelyn hand soap gardeners 10 1 f...
4,B00005NDTD,love this lotion,four star,remember why pron love pron favorite blanket t...,soy milk hand crme
...,...,...,...,...,...
12106,B01HIQEOLO,color be perfect go on smooth,five star,cnd shellac be design to be use as a system fe...,cnd shellac leather satchel
12107,B01HIQHQU0,not impressed with this very dull gray blue co...,blah,cnd shellac be design to be use as a system fe...,cnd shellac power polish denim patch
12108,B01HIQIEYC,pron receive lot of compliment when pron wear...,fantastic color,cnd craft culture collection patina buckle di...,cnd shellac power polish patina buckle
12109,B01HJ2UY0W,the perfume be good but the spray head break off,spray head break off within a month,the pron am juicy couture girl be once again t...,juicy couture pron love juicy couture 1 7 fl o...


#Способы для поиска упоминания товаров в отзывах

1. Можно использовать шаблоны, задающие синтаксические и грамматические правила, основывающиеся на частях речи: последовтаельности определенных типов объектов.
2. Поиск по n-граммам путем формирования шаблонов, содержащих в себе категории. К сожалению, категории очень часто оказываются пустыми, поэтому работать с ними - не лучшая идея. Но еще нужно будет посмотреть на сами категории: что они из себя представляют, какой ассортимент охватывают и тд.
3. Создание словарей с ключевыми словами товаров из пересечения *description* и *title*. Если ключевое слово *description* есть в *title* - включаем его в наш словарь. Словарь можно дополнить путем использования эмбеддингов: ищем похожие слова и добавляем в словарь. Тут могут возникнуть проблемы с ключевыми словами и частотностью отдельных слов в целом.

#Словарь с ключевыми словами

In [ ]:
 vectorizer = TfidfVectorizer()

В прошлой домашней работе Tf-Idf получше других справлялся с выделением ключевых слов. Возьмем его.

In [ ]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

In [222]:
def tfidf_keywords(text):
  if len(text) <= 1:
    return 'NO KEYWORDS'
  else:
    X = vectorizer.fit_transform(text)
    feature_names = np.array(vectorizer.get_feature_names())
    denselist = np.array(X.todense())
    indices = denselist.argsort()[0]
    text_keywords = list(feature_names[indices[::-1][:20]])
    return text_keywords

In [223]:
tfidf_keywords_list = []
for text in tqdm(data_preprocessed['reviewText']):
  tfidf_keywords_list.append(tfidf_keywords(text.split()))

100%|██████████| 12111/12111 [00:21<00:00, 571.05it/s]


In [226]:
df_analysis = pd.DataFrame()
df_analysis['title'] = data_preprocessed['title']
df_analysis['keywords'] = tfidf_keywords_list
df_analysis = df_analysis.loc[df_analysis['keywords'] != 'NO KEYWORDS'].reset_index()
df_analysis = df_analysis.drop(columns='index')
df_analysis

,title,keywords
0,crabtree amp evelyn gardener s ultra moisturis...,"[pron, work, do, neither, much, hand, get, fl,..."
1,ahava bath salts,"[pron, will, could, have, great, good, give, f..."
2,ahava dead sea mineral mud 8 5 oz pack of 4,"[pron, weird, people, add, and, anymore, be, h..."
3,crabtree amp evelyn hand soap gardeners 10 1 f...,"[this, wrong, evelyn, house, hope, have, hand,..."
4,soy milk hand crme,"[love, this, lotion]"
...,...,...
11823,cnd shellac leather satchel,"[color, smooth, perfect, on, go, be]"
11824,cnd shellac power polish denim patch,"[not, with, impressed, blue, but, cnd, color, ..."
11825,cnd shellac power polish patina buckle,"[pron, which, lot, and, angle, at, color, comp..."
11826,juicy couture pron love juicy couture 1 7 fl o...,"[the, spray, perfume, off, head, good, but, br..."


In [229]:
kw_freq = {}
for i in range(len(df_analysis)):
  for kw in df_analysis['keywords'][i]:
    if kw in df_analysis['title'][i].split():
      if kw in kw_freq.keys():
        kw_freq[kw] += 1
      else:
        kw_freq[kw] = 1

In [234]:
sorted_kw = sorted(kw_freq.items(), key=lambda kv: kv[1], reverse=True)
sorted_kw

[('hair', 446),
 ('for', 367),
 ('cream', 234),
 ('and', 231),
 ('eye', 170),
 ('brush', 161),
 ('lip', 139),
 ('pron', 138),
 ('shampoo', 129),
 ('color', 125),
 ('conditioner', 118),
 ('hand', 110),
 ('gel', 105),
 ('nail', 103),
 ('body', 97),
 ('skin', 92),
 ('face', 86),
 ('lotion', 85),
 ('the', 71),
 ('in', 68),
 ('cleanser', 66),
 ('iron', 65),
 ('dry', 59),
 ('polish', 57),
 ('dryer', 52),
 ('candle', 51),
 ('oil', 51),
 ('be', 50),
 ('foundation', 50),
 ('mask', 50),
 ('soap', 48),
 ('with', 48),
 ('spray', 40),
 ('mascara', 39),
 ('curl', 39),
 ('of', 38),
 ('gloss', 37),
 ('moisturizer', 35),
 ('blush', 35),
 ('concealer', 35),
 ('balm', 32),
 ('lash', 31),
 ('lipstick', 31),
 ('lavender', 29),
 ('iredale', 28),
 ('hold', 28),
 ('zoya', 28),
 ('elemis', 27),
 ('occitane', 27),
 ('eyeliner', 27),
 ('wash', 26),
 ('kit', 26),
 ('jane', 25),
 ('facial', 25),
 ('black', 25),
 ('oz', 25),
 ('powder', 24),
 ('butter', 24),
 ('evelyn', 23),
 ('crabtree', 23),
 ('makeup', 23),
 ('b

In [235]:
lux_list = ['cream', 'brush', 'shampoo', 'conditioner', 'gel', 'lotion', 'cleanser', 
            'polish', 'dryer', 'candle', 'oil', 'foundation', 'mask', 'soap', 
            'spray', 'mascara', 'gloss', 'moisturizer', 'blush', 'concealer', 'balm']

Кажется, что это наиболее лучшие ключевые слова для дальнейшей работы. У них довольно большая частотность, но в целом в словаре довольно много хороших слов.

In [237]:
ft = gensim.downloader.load('fasttext-wiki-news-subwords-300')

[==================================================] 100.0% 958.5/958.4MB downloaded


In [245]:
full_lux = []
for lux in lux_list:
  if lux not in full_lux:
    full_lux.append(lux)
    index = 0
    if lux in ft:
      while index != -1:
        simmilar = ft.most_similar(lux)[index][0]
        if lux == preprocessing(simmilar):
          index += 1
        else:
          full_lux.append(simmilar)
          index = -1

In [246]:
full_lux = set(full_lux)

#Биграммы

In [281]:
bigrams = []

for i, review in enumerate(tqdm(data_preprocessed['reviewText'])):
  review = review.split()
  for lux in full_lux:
    line = []
    for i, word in enumerate(review):
      if word == lux:
        line.append(i)
    if line:
      for i in line:
        if i != 0:
          bigrams.append(review[(i - 1):(i + 1)])
        if i != len(review) - 1:
          bigrams.append(review[i:(i + 2)])

100%|██████████| 12111/12111 [00:03<00:00, 4032.73it/s]


In [282]:
bigrams

[['hand', 'cream'],
 ['cream', 'and'],
 ['hand', 'cream'],
 ['cream', 'be'],
 ['good', 'soap'],
 ['soap', 'for'],
 ['evelyn', 'soap'],
 ['soap', 'for'],
 ['pump', 'soap'],
 ['soap', 'could'],
 ['wrong', 'soap'],
 ['soap', 'hope'],
 ['this', 'lotion'],
 ['different', 'lotion'],
 ['lotion', 'but'],
 ['generic', 'moisturizer'],
 ['moisturizer', 'for'],
 ['regular', 'moisturizer'],
 ['moisturizer', 'pron'],
 ['try', 'cream'],
 ['cream', 'natural'],
 ['mineral', 'cream'],
 ['cream', 'yoga'],
 ['blow', 'dryer'],
 ['dryer', 'speed'],
 ['counter', 'cream'],
 ['cream', 'pron'],
 ['nail', 'polish'],
 ['polish', 'once'],
 ['nail', 'polish'],
 ['polish', 'pron'],
 ['this', 'polish'],
 ['polish', 'under'],
 ['this', 'polish'],
 ['polish', 'pron'],
 ['good', 'polish'],
 ['polish', 'pron'],
 ['this', 'polish'],
 ['polish', 'as'],
 ['glitter', 'polish'],
 ['polish', 'this'],
 ['other', 'polish'],
 ['polish', 'but'],
 ['the', 'mascara'],
 ['mascara', 'when'],
 ['night', 'cream'],
 ['cream', 'pron'],
 [

In [283]:
bigram_scores = nltk.collocations.BigramAssocMeasures()
bigrams_finder = nltk.collocations.BigramCollocationFinder.from_documents(bigrams)
bigrams_finder.apply_freq_filter(10)

In [284]:
DICE = bigrams_finder.score_ngrams(bigram_scores.dice)
PMI = bigrams_finder.score_ngrams(bigram_scores.pmi)
CHI_SQ = bigrams_finder.score_ngrams(bigram_scores.chi_sq)

In [285]:
for i in DICE[:20]:
  print(i)

(('lip', 'gloss'), 0.30564784053156147)
(('lip', 'balm'), 0.2973621103117506)
(('nail', 'polish'), 0.288864388092613)
(('hair', 'dryer'), 0.21267893660531698)
(('blow', 'dryer'), 0.19950124688279303)
(('shower', 'gel'), 0.13617021276595745)
(('this', 'shampoo'), 0.13403736799350122)
(('shampoo', 'and'), 0.11496196111580727)
(('the', 'brush'), 0.10077519379844961)
(('and', 'conditioner'), 0.09075997813012575)
(('the', 'shampoo'), 0.08653846153846154)
(('brush', 'be'), 0.08579335793357934)
(('cream', 'pron'), 0.08576642335766424)
(('hair', 'spray'), 0.08459214501510574)
(('this', 'lotion'), 0.08382642998027613)
(('this', 'cream'), 0.0806345009914078)
(('the', 'lotion'), 0.07759456838021339)
(('brush', 'pron'), 0.07692307692307693)
(('lotion', 'pron'), 0.07510917030567686)
(('a', 'moisturizer'), 0.07370184254606366)


Выглядит довольно хорошо

In [286]:
for j in PMI[:20]:
  print(j)

(('blow', 'dryer'), 6.329532019050701)
(('lip', 'gloss'), 5.986644305527687)
(('mud', 'mask'), 5.7980026299099805)
(('lip', 'balm'), 5.712021925418684)
(('liquid', 'foundation'), 5.420297482973879)
(('shower', 'gel'), 5.292541935775509)
(('nail', 'polish'), 5.1780662422006465)
(('gentle', 'cleanser'), 5.0804264518483055)
(('hair', 'dryer'), 5.054368391499256)
(('polish', 'remover'), 5.039461003971139)
(('essential', 'oil'), 4.894113019773833)
(('tint', 'moisturizer'), 4.733534226866951)
(('an', 'oil'), 4.701467941831439)
(('coconut', 'oil'), 4.653104920270035)
(('spray', 'bottle'), 4.434696862034498)
(('pearson', 'brush'), 4.2976319345844125)
(('bristle', 'brush'), 4.1049868566420145)
(('facial', 'cleanser'), 4.080426451848304)
(('badger', 'brush'), 4.034597528750618)
(('brush', 'head'), 4.034597528750618)


Здесь тоже неплохо, не особо отличается от DICE.

In [287]:
for k in CHI_SQ[:20]:
  print(k)

(('nail', 'polish'), 4625.101364903197)
(('blow', 'dryer'), 3180.329162880735)
(('lip', 'balm'), 3171.6100744330856)
(('lip', 'gloss'), 2854.480090067156)
(('shower', 'gel'), 1829.9087882966294)
(('hair', 'dryer'), 1652.6143803180616)
(('mud', 'mask'), 873.7708306851434)
(('eye', 'cream'), 752.2698706905038)
(('liquid', 'foundation'), 665.318034731522)
(('hand', 'cream'), 561.4592001713944)
(('polish', 'remover'), 539.9108160511395)
(('tint', 'moisturizer'), 457.3340564738385)
(('night', 'cream'), 451.494741347947)
(('hair', 'spray'), 402.8634386389117)
(('gentle', 'cleanser'), 357.1692842246559)
(('an', 'oil'), 348.7062262178917)
(('essential', 'oil'), 344.965143879993)
(('spray', 'bottle'), 344.7709398569641)
(('facial', 'cleanser'), 336.35732079246253)
(('body', 'lotion'), 331.4042822007234)


Они в принципе все похожи.

#Группировка коллокаций (биграммы)

In [296]:
collocations_bigrams = {}
for lux in full_lux:
  collocations_bigrams[lux] = []
for bigram in DICE:
  words = set(list(bigram[0])) & full_lux
  for word in words:
    collocations_bigrams[word].append(' '.join(bigram[0]))
for lux in full_lux:
  collocations_bigrams[lux] = list(set(collocations_bigrams[lux]))
for word in ['cream', 'brush', 'shampoo', 'conditioner', 'gel', 'lotion', 'cleanser']:
  print(word)
  for i in collocations_bigrams[word]:
    print('\t'+i)

cream
	body cream
	cream do
	cream blush
	a cream
	lotion cream
	bb cream
	cream on
	cream be
	shaving cream
	cream the
	cream which
	neck cream
	cream have
	cream for
	cream but
	cream with
	day cream
	shave cream
	night cream
	the cream
	this cream
	and cream
	cc cream
	cream this
	cream to
	cream that
	cream in
	eye cream
	cream and
	cream pron
	face cream
	hand cream
brush
	brush for
	and brush
	brush will
	bristle brush
	blush brush
	brush and
	pearson brush
	these brush
	brush be
	brush on
	a brush
	brush from
	badger brush
	brush the
	brush pron
	brush head
	this brush
	the brush
	brush but
	brush in
	foundation brush
	great brush
	good brush
	to brush
	brush do
	brush to
	brush have
	pron brush
	brush that
shampoo
	other shampoo
	shampoo and
	shampoo leave
	a shampoo
	pron shampoo
	of shampoo
	shampoo conditioner
	great shampoo
	shampoo but
	shampoo pron
	good shampoo
	shampoo to
	shampoo in
	the shampoo
	shampoo with
	shampoo do
	shampoo be
	this shampoo
	shampoo for
	to shamp

В целом особых различий между DICE, PMI и CHI_SQ я не заметила. На мой взгляд DICE работает немного лучше, если посмотреть, например, на cleanser. Особенно для cream он выделяет неплохие сочетания: назначения, разные прилагательные отношения к продукты, прилагательные, относящиеся к описанию самого продукта и тд.

#Триграммы

In [297]:
trigrams = []

for i, review in enumerate(tqdm(data_preprocessed['reviewText'])):
  review = review.split()
  for lux in full_lux:
    line = []
    for i, word in enumerate(review):
      if word == lux:
        line.append(i)
    if line:
      for i in line:
        if i != 0:
          trigrams.append(review[(i - 1):(i + 2)])

100%|██████████| 12111/12111 [00:02<00:00, 4038.84it/s]


In [298]:
trigrams

[['hand', 'cream', 'and'],
 ['hand', 'cream', 'be'],
 ['good', 'soap', 'for'],
 ['evelyn', 'soap', 'for'],
 ['pump', 'soap', 'could'],
 ['wrong', 'soap', 'hope'],
 ['this', 'lotion'],
 ['different', 'lotion', 'but'],
 ['generic', 'moisturizer', 'for'],
 ['regular', 'moisturizer', 'pron'],
 ['try', 'cream', 'natural'],
 ['mineral', 'cream', 'yoga'],
 ['blow', 'dryer', 'speed'],
 ['counter', 'cream', 'pron'],
 ['nail', 'polish', 'once'],
 ['nail', 'polish', 'pron'],
 ['this', 'polish', 'under'],
 ['this', 'polish', 'pron'],
 ['good', 'polish', 'pron'],
 ['this', 'polish', 'as'],
 ['glitter', 'polish', 'this'],
 ['other', 'polish', 'but'],
 ['the', 'mascara', 'when'],
 ['night', 'cream', 'pron'],
 ['from', 'foundation', 'up'],
 ['silkcoat', 'balm', 'pron'],
 ['silkcoat', 'balm', 'only'],
 ['silkcoat', 'balm', 'pron'],
 ['good', 'lotion', 'ever'],
 ['of', 'foundation', 'and'],
 ['pron', 'cleanser', 'this'],
 ['this', 'soap', 'be'],
 ['too', 'soap', 'disappear'],
 ['novelty', 'soap'],
 ['al

In [299]:
trigrams_scores = nltk.collocations.TrigramAssocMeasures()
trigrams_finder = nltk.collocations.TrigramCollocationFinder.from_documents(trigrams)
trigrams_finder.apply_freq_filter(5)

In [300]:
JAC = trigrams_finder.score_ngrams(trigrams_scores.jaccard)
PMI = trigrams_finder.score_ngrams(trigrams_scores.pmi)
CHI_SQ = trigrams_finder.score_ngrams(trigrams_scores.chi_sq)

In [301]:
for l in JAC[:20]:
  print(l)

(('nail', 'polish', 'remover'), 0.026829268292682926)
(('lip', 'gloss', 'do'), 0.017064846416382253)
(('the', 'brush', 'be'), 0.015341701534170154)
(('the', 'shampoo', 'and'), 0.014705882352941176)
(('this', 'shampoo', 'be'), 0.012493993272465162)
(('nail', 'polish', 'but'), 0.011965811965811967)
(('nail', 'polish', 'pron'), 0.010309278350515464)
(('lip', 'balm', 'pron'), 0.0101763907734057)
(('the', 'lotion', 'be'), 0.00973360655737705)
(('this', 'shampoo', 'pron'), 0.009242144177449169)
(('nail', 'polish', 'and'), 0.00909090909090909)
(('this', 'shampoo', 'for'), 0.008214676889375685)
(('a', 'spray', 'bottle'), 0.007978723404255319)
(('hair', 'spray', 'or'), 0.007961783439490446)
(('lip', 'balm', 'and'), 0.007936507936507936)
(('this', 'brush', 'be'), 0.007925407925407926)
(('this', 'lotion', 'be'), 0.007861635220125786)
(('nail', 'polish', 'be'), 0.00784313725490196)
(('blow', 'dryer', 'pron'), 0.007586206896551724)
(('a', 'moisturizer', 'that'), 0.007583965330444204)


In [302]:
for m in PMI[:20]:
  print(m)

(('nail', 'polish', 'remover'), 12.272729701741241)
(('lip', 'gloss', 'do'), 10.742825903605056)
(('good', 'lotion', 'ever'), 9.692552697123826)
(('a', 'spray', 'bottle'), 9.616876767523873)
(('blow', 'dryer', 'pron'), 9.159981826968046)
(('zoya', 'polish', 'pron'), 9.073326572933443)
(('hair', 'spray', 'or'), 8.6960116409894)
(('lip', 'gloss', 'be'), 8.453812431190428)
(('nail', 'polish', 'but'), 8.357618599327754)
(('lip', 'balm', 'pron'), 8.337086133364501)
(('lip', 'balm', 'and'), 8.065255341917634)
(('tint', 'moisturizer', 'pron'), 8.027959874815597)
(('the', 'spray', 'bottle'), 7.878563489895921)
(('lip', 'balm', 'be'), 7.751013440175413)
(('lip', 'gloss', 'the'), 7.733365574355986)
(('lip', 'gloss', 'pron'), 7.717957029492155)
(('shower', 'gel', 'be'), 7.653558565507968)
(('shower', 'gel', 'and'), 7.605230387865486)
(('neck', 'cream', 'pron'), 7.4022443974688805)
(('hair', 'dryer', 'and'), 7.319364682302528)


In [303]:
for n in CHI_SQ[:20]:
  print(n)

(('nail', 'polish', 'remover'), 60234.638012952615)
(('lip', 'gloss', 'do'), 11959.775876733127)
(('blow', 'dryer', 'pron'), 8968.453874058307)
(('nail', 'polish', 'but'), 8438.393322649747)
(('lip', 'balm', 'pron'), 7724.66132810005)
(('nail', 'polish', 'pron'), 7475.100116893369)
(('nail', 'polish', 'and'), 7220.9656353261225)
(('nail', 'polish', 'be'), 7091.563071370389)
(('lip', 'balm', 'and'), 6012.219577860718)
(('lip', 'gloss', 'be'), 5233.910967221944)
(('lip', 'balm', 'be'), 5226.005105534384)
(('a', 'spray', 'bottle'), 4931.456077836497)
(('good', 'lotion', 'ever'), 4781.856972495981)
(('lip', 'gloss', 'pron'), 4696.18977358578)
(('lip', 'gloss', 'the'), 4607.762619274542)
(('shower', 'gel', 'and'), 3367.9305566577536)
(('shower', 'gel', 'be'), 3196.44343727495)
(('hair', 'dryer', 'pron'), 3185.4774813407703)
(('hair', 'dryer', 'and'), 3090.15141377311)
(('shower', 'gel', 'pron'), 2892.9142685872785)


#Группировка коллокаций (триграммы)

In [304]:
collocations_trigrams = {}
for lux in full_lux:
  collocations_trigrams[lux] = []
for trigram in PMI:
  words = set(list(trigram[0])) & full_lux
  for word in words:
    collocations_trigrams[word].append(' '.join(trigram[0]))
for lux in full_lux:
  collocations_trigrams[lux] = list(set(collocations_trigrams[lux]))
for word in ['cream', 'brush', 'shampoo', 'conditioner', 'gel', 'lotion', 'cleanser']:
  print(word)
  for i in collocations_trigrams[word]:
    print('\t'+i)

cream
	this cream have
	eye cream be
	this cream on
	this cream for
	this cream be
	this cream pron
	shave cream and
	eye cream and
	this cream do
	neck cream pron
	shaving cream pron
	a cream pron
	shave cream pron
	night cream pron
	shaving cream and
	the cream be
	a cream that
	hand cream be
	eye cream pron
	shave cream be
	the cream pron
	face cream pron
	the cream and
	hand cream pron
	night cream and
	the cream have
brush
	pron brush pron
	this brush be
	pron brush and
	the brush and
	the brush be
	the brush pron
	the brush that
	to brush pron
	this brush pron
shampoo
	this shampoo have
	the shampoo be
	this shampoo and
	the shampoo pron
	the shampoo and
	this shampoo work
	this shampoo pron
	dry shampoo pron
	this shampoo do
	this shampoo for
	this shampoo be
	good shampoo pron
	the shampoo conditioner
conditioner
	this conditioner be
	the shampoo conditioner
	this conditioner pron
	the conditioner pron
	and conditioner pron
	in conditioner pron
	the conditioner be
gel
	shower g

Здесь JAC справился не очень хорошо, остальные примерно одинаково, но мне все-таки результаты PMI показались получше. Опять же можно наблюдать лучшие результаты на cream. Выделяет триграммы с назначением, отношением к продукту, но довольно много предлогов и прочих таких вещей. Еще выделяет вид продукта (например ночной крем).